In [1]:
# Library imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
import math
from math import e
from tqdm import tqdm
import gc

# Importing model, data loaders and other utilities
from models.neo_phurie import NeoPHURIE
from dataLoaders.image_loaders import ImageDataset, ImageDataset_Testing, LABELS_STD
from utils import EpsilonInsensitiveLoss, OneCycleLR

# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
DATASET_PATH = "./datasets/images_dataset/"

## Model Training

In [2]:
def train_model(model, 
          num_epochs,
          start_learning_rate = 1e-5,
          max_learning_rate = 1e-4, 
          regularization_rate = 0, 
          train_years = [],
          verbose = False):
    
    # Memory management
    gc.collect()
    torch.cuda.empty_cache()

    # Training Dataset
    train_loader = DataLoader(ImageDataset(DATASET_PATH, train_years, transform=transforms.Resize(224), standardization=True), 
                              batch_size=BATCH_SIZE, 
                              shuffle=True, 
                              num_workers=4)

    # Training optimizer, scheduler and loss
    optimizer = optim.SGD(model.parameters(), lr=start_learning_rate, weight_decay=regularization_rate)
    scheduler = OneCycleLR(optimizer, num_steps = num_epochs*len(train_loader), lr_range=(start_learning_rate, max_learning_rate))
    criterion = EpsilonInsensitiveLoss(0.2, 2)

    # Scaler for mixed-precision training
    scaler = torch.cuda.amp.GradScaler()

    # Moving model to compute device
    model.to(device)

    # Training loop
    for epoch in range(num_epochs):

        # Training Phase
        model.train()
        train_squared_errors = 0.0

        # Progress bar
        pbar = tqdm(enumerate(train_loader, 0), total = len(train_loader), unit = "images", unit_scale=BATCH_SIZE, disable=not verbose)
        
        # Iterate through batches in training dataset
        for i, data in pbar:
            inputs, labels = data

            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            # Display MSE and RMSE on progress bar
            train_squared_errors += nn.functional.mse_loss(outputs, labels.to(device)).item() * (LABELS_STD**2) * BATCH_SIZE
            pbar.set_description(f"[Epoch {epoch + 1}]: Training Loss = {train_squared_errors / ((i + 1) * BATCH_SIZE):.4f}: Training RMSE = {math.sqrt(train_squared_errors / ((i + 1) * BATCH_SIZE)):.4f}")

        # Computing final training MSE over entire epoch
        train_mse = train_squared_errors / len(train_loader.dataset)

        if verbose:
            print(f"[Epoch {epoch + 1}]: Training Loss = {train_mse:.4f}; Training RMSE = {math.sqrt(train_mse):.4f}")

        # Memory management
        gc.collect()
        torch.cuda.empty_cache()


## Model Testing

In [3]:
def test_model(model, testing_years):

    def weighted_average_smoothing(predictions):
        """ Method to apply temporal averaging post-processing on model predictions. """
        out = torch.zeros(predictions.shape)

        for i in range(out.shape[0]):
            coef = 1 / sum(e ** (np.array(range(0, min(i + 1, 6))) * -0.5))

            for j in range(max(i - 5, 0), i+1):
                out[i][0] += predictions[j][0] * coef * (e ** (-(i - j) / 2))

        return out

    # Testing dataset
    test_dataset = ImageDataset_Testing(DATASET_PATH, testing_years, transforms.Resize(224), standardization=True)

    # Moving model to compute device
    model.to(device)

    test_squared_errors = torch.tensor(0.0)
    errors = []
    
    for hurricane in test_dataset:
        with torch.no_grad():
            model.eval()
            inputs, labels = hurricane

            with autocast():
                outputs = model(inputs.to(device))
                smoothed_outputs = weighted_average_smoothing(outputs.cpu())
                test_squared_errors += (smoothed_outputs - labels).square().sum()
                
            errors += ((smoothed_outputs - labels) * LABELS_STD).abs().tolist()

            # Memory management
            torch.cuda.empty_cache()

    test_rmse = (test_squared_errors / test_dataset.get_nr_images()).sqrt()
    
    return test_rmse.item() * LABELS_STD, np.quantile(errors, 0.25), np.quantile(errors, 0.5), np.quantile(errors, 0.75)

### Example Usage

In [4]:
model = NeoPHURIE()

train_model(model,
      num_epochs = 4,
      start_learning_rate = 6e-3,
      max_learning_rate = 6e-2,
      regularization_rate = 1e-5,
      train_years = range(2001, 2015),
      verbose = True)

rmse, q1, q2, q3 = test_model(model, [2015])
print(f"Test results: RMSE = {rmse:.2f} ; Q1 = {q1:.2f} ; Median = {q2:.2f} ; Q3 = {q3:.2f}")

[Epoch 1]: Training Loss = 241.1904: Training RMSE = 15.5303: 100%|██████████| 102144/102144 [01:49<00:00, 929.63images/s] 


[Epoch 1]: Training Loss = 241.2589; Training RMSE = 15.5325


[Epoch 2]: Training Loss = 161.2088: Training RMSE = 12.6968: 100%|██████████| 102144/102144 [01:42<00:00, 999.97images/s] 


[Epoch 2]: Training Loss = 161.2546; Training RMSE = 12.6986


[Epoch 3]: Training Loss = 120.3663: Training RMSE = 10.9712: 100%|██████████| 102144/102144 [01:45<00:00, 968.22images/s]


[Epoch 3]: Training Loss = 120.4004; Training RMSE = 10.9727


[Epoch 4]: Training Loss = 85.8399: Training RMSE = 9.2650: 100%|██████████| 102144/102144 [01:47<00:00, 953.22images/s]


[Epoch 4]: Training Loss = 85.8642; Training RMSE = 9.2663
Test results: RMSE = 9.41 ; Q1 = 2.11 ; Median = 4.60 ; Q3 = 8.96


### Testing LOYO performance across all years

In [4]:
for test_year in range(2001, 2016):
    model = NeoPHURIE()
    
    train_model(model,
      num_epochs = 4,
      start_learning_rate = 6e-3,
      max_learning_rate = 6e-2,
      regularization_rate = 1e-5,
      train_years = [y for y in range(2001, 2016) if y != test_year],
      verbose = False)
    
    rmse, q1, q2, q3 = test_model(model, [test_year])
    print(f"LOYO year {test_year}: RMSE = {rmse:.4f} ; Q1 = {q1:.4f} ; Median = {q2:.4f} ; Q3 = {q3:.4f}")

LOYO year 2001: RMSE = 8.7412 ; Q1 = 2.1003 ; Median = 4.6629 ; Q3 = 8.8653
LOYO year 2002: RMSE = 7.5954 ; Q1 = 1.9732 ; Median = 4.3643 ; Q3 = 8.4156
LOYO year 2003: RMSE = 8.0649 ; Q1 = 2.0839 ; Median = 4.5311 ; Q3 = 8.2166
LOYO year 2004: RMSE = 8.3169 ; Q1 = 2.1315 ; Median = 4.6315 ; Q3 = 8.2638
LOYO year 2005: RMSE = 9.0331 ; Q1 = 2.2656 ; Median = 4.6829 ; Q3 = 8.8538
LOYO year 2006: RMSE = 8.2845 ; Q1 = 1.9923 ; Median = 4.2790 ; Q3 = 7.9773
LOYO year 2007: RMSE = 8.2793 ; Q1 = 2.1441 ; Median = 4.6410 ; Q3 = 8.5142
LOYO year 2008: RMSE = 8.0887 ; Q1 = 1.9185 ; Median = 4.3943 ; Q3 = 8.4330
LOYO year 2009: RMSE = 7.8678 ; Q1 = 2.2430 ; Median = 4.6450 ; Q3 = 7.8896
LOYO year 2010: RMSE = 8.4903 ; Q1 = 2.1223 ; Median = 4.9156 ; Q3 = 9.0567
LOYO year 2011: RMSE = 7.6239 ; Q1 = 2.0982 ; Median = 4.7507 ; Q3 = 8.3725
LOYO year 2012: RMSE = 7.4892 ; Q1 = 1.8036 ; Median = 4.0995 ; Q3 = 7.3812
LOYO year 2013: RMSE = 7.7556 ; Q1 = 2.0000 ; Median = 4.4084 ; Q3 = 7.9886
LOYO year 20